In [1]:
import argparse
import cv2
import numpy as np
from pathlib import Path
from PIL import Image
import math
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm
from natsort import natsorted

In [18]:
def parse_args():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('--input_channels', default=3, type=int,
                        help='input channels')
    parser.add_argument('--original_w', default=630, type=int,
                        help='image width')
    parser.add_argument('--original_h', default=910, type=int,
                        help='image height')
    parser.add_argument('--resize_w', default=224, type=int,
                        help='image width')
    parser.add_argument('--resize_h', default=224, type=int,
                        help='image height')
    
    # gaus filter
    parser.add_argument('--kernel_size', default=201, type=int,
                        help='image width')
    parser.add_argument('--sigma', default=6, type=int,
                        help='image height')
    
    # dataset
    parser.add_argument('--coord_list', default='coord_list.csv',
                        help='coord_list')
    parser.add_argument('--img_ext', default='.png',
                        help='image file extension')
    parser.add_argument('--num_coord', default=15, type=int,
                        help='number of coordinate')
    
    parser.add_argument('--dir_path', default='data_set/', type=str)
    parser.add_argument('--dir_name', default='images', type=str)
    
    config = parser.parse_args(args=['--dir_name', 'images'])

    return config

In [19]:
def gaus_filter(img, kernel_size, sigma):
    pad_size = int(kernel_size - 1 / 2)
    img_t = np.pad(
        img, (pad_size, pad_size), "constant"
    )  # zero padding
    img_t = cv2.GaussianBlur(
        img_t, ksize=(kernel_size, kernel_size), sigmaX=sigma
    )  # gaussian filter 
    img_t = img_t[pad_size:-pad_size, pad_size:-pad_size]  # remove padding
    return img_t

In [20]:
def main():
    config = vars(parse_args())
    
    for i in range (config['num_coord']):   
        os.makedirs(config['dir_path'] + "/" + "/masks/" + f'{i}', exist_ok=True)
        
    img_list =[]    
    for curDir, dirs, files in tqdm(natsorted(os.walk(config['dir_path'] + "/" + config['dir_name']))):
        for i, file in enumerate(files):
            if file.endswith(config['img_ext']):
                img_list.append(file)
                
    coord_list = pd.read_csv(config['dir_path'] + "/" + config['coord_list'])
    img_list = pd.DataFrame(img_list).set_axis([coord_list.columns[0]], axis='columns')        
    img_coord_df = pd.merge(img_list, coord_list)
        
    #Extract the x- and y-coordinates     
    dfx= coord_list.iloc[0: ,1:16]
    dfy= coord_list.iloc[0: ,16:31]
  
    x_columns=dfx.columns
    y_columns=dfy.columns
    columns=[]
    for i in range (len(x_columns)):
        name=x_columns[i].split('_')[0]
        columns.append(name)

    ptarray_x=np.array(dfx)
    ptarray_y=np.array(dfy)

    ptarray_stdx=(ptarray_x/config['original_w'])*config['resize_w']
    ptarray_stdy=(ptarray_y/config['original_h'])*config['resize_h']
    dfx = pd.DataFrame(ptarray_stdx)
    dfy = pd.DataFrame(ptarray_stdy)      
        
    # make back ground
    black = np.zeros((config['resize_w'], config['resize_h']))

    for i in range(len(coord_list)):
        for j in range(config['num_coord']):
            result = black.copy()
            img_t = black.copy() 
            img_t[int(dfy.iloc[i][j])][int(dfx.iloc[i][j])] = 255
            img_t = gaus_filter(img_t, 301, 5)
            result = np.maximum(result, img_t) 

            #  normalization
            result = 255 * result / result.max()
            result = result.astype("uint8")
            cv2.imwrite(config['dir_path'] + "/"  +'masks' + "/" +f'{j}' + "/" +f'{coord_list[coord_list.columns[0]][i]}', result)
        print(i + 1)
    print("finish")
if __name__ == '__main__':
    main()

100%|██████████| 1/1 [00:00<?, ?it/s]


1
2
3
4
5
6
7
8
9
10
11
12
finish
